In [11]:
import simpy
import random

# Constants
AVG_ARRIVAL_TIME = 5 * 60  # Average time between vessel arrivals (5 hours in minutes)
NUM_CONTAINERS = 150  # Number of containers per vessel
CRANE_TIME_PER_CONTAINER = 3  # Time to move one container (in minutes)
TRUCK_TIME = 6  # Time for a truck to drop off a container and come back (in minutes)
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of trucks
SIMULATION_TIME = 24 * 60  # Simulation time in minutes (24 hours)

# Initialize the random seed for reproducibility
random.seed(42)

class Terminal:
    def __init__(self, env):
        self.env = env
        self.berths = simpy.Resource(env, NUM_BERTHS)
        self.cranes = simpy.Resource(env, NUM_CRANES)
        self.trucks = simpy.Resource(env, NUM_TRUCKS)

    def berth_vessel(self, vessel):
        with self.berths.request() as request:
            yield request
            print(f"Time {self.env.now}: Vessel {vessel.id} has berthed.")
            yield self.env.process(self.unload_vessel(vessel))

    def unload_vessel(self, vessel):
        for _ in range(NUM_CONTAINERS):
            with self.cranes.request() as crane_request:
                yield crane_request
                yield self.env.timeout(CRANE_TIME_PER_CONTAINER)
                print(f"Time {self.env.now}: Crane has moved a container from Vessel {vessel.id}.")
                with self.trucks.request() as truck_request:
                    yield truck_request
                    yield self.env.timeout(TRUCK_TIME)
                    print(f"Time {self.env.now}: Truck has delivered a container from Vessel {vessel.id}.")
        print(f"Time {self.env.now}: Vessel {vessel.id} has finished unloading and is leaving.")

class Vessel:
    def __init__(self, env, terminal, id):
        self.env = env
        self.terminal = terminal
        self.id = id
        self.action = env.process(self.arrive())

    def arrive(self):
        print(f"Time {self.env.now}: Vessel {self.id} is arriving.")
        yield self.env.process(self.terminal.berth_vessel(self))

def vessel_arrivals(env, terminal):
    vessel_id = 1
    while True:
        arrival_time = random.expovariate(1.0 / AVG_ARRIVAL_TIME)
        yield env.timeout(arrival_time)
        vessel = Vessel(env, terminal, vessel_id)
        vessel_id += 1

def run_simulation(simulation_time):
    env = simpy.Environment()
    terminal = Terminal(env)
    env.process(vessel_arrivals(env, terminal))
    env.run(until=simulation_time)

# Run the simulation
run_simulation(SIMULATION_TIME)


Time 306.0180861824403: Vessel 1 is arriving.
Time 306.0180861824403: Vessel 1 has berthed.
Time 309.0180861824403: Crane has moved a container from Vessel 1.
Time 313.61673789526196: Vessel 2 is arriving.
Time 313.61673789526196: Vessel 2 has berthed.
Time 315.0180861824403: Truck has delivered a container from Vessel 1.
Time 316.61673789526196: Crane has moved a container from Vessel 2.
Time 318.0180861824403: Crane has moved a container from Vessel 1.
Time 322.61673789526196: Truck has delivered a container from Vessel 2.
Time 324.0180861824403: Truck has delivered a container from Vessel 1.
Time 325.61673789526196: Crane has moved a container from Vessel 2.
Time 327.0180861824403: Crane has moved a container from Vessel 1.
Time 331.61673789526196: Truck has delivered a container from Vessel 2.
Time 333.0180861824403: Truck has delivered a container from Vessel 1.
Time 334.61673789526196: Crane has moved a container from Vessel 2.
Time 336.0180861824403: Crane has moved a container 